# This Notebook has all of the code samples shown in the presentation.
eNBy is aliased to nblib since it sounded more professional, though most of the code is from other scripts put into a notebook.

In [ ]:
from typing import List
import netsblox

client = netsblox.Client(run_forever=True)
roboscape = netsblox.RoboScape(client)

commandStorage: List[str] = []
robots: List[str] = roboscape.get_robots()


@client.on_message("robot command")
def on_command(robot, command):
    tempStorage: str = f"{robot}: {command}"
    commandStorage.append(tempStorage)
    if len(commandStorage) > 99:
        commandStorage.pop(0)


for robot in robots:
    roboscape.listen(robot)

In [5]:
import netsblox

client = netsblox.Client(run_forever=True)
roboscape = netsblox.RoboScape(client)

for robot in roboscape.get_robots():
    if robot.endswith('37ac'): continue
    roboscape.send(robot, 'beep 100 100')
    print(f'Beeped {robot}')

exit()

Beeped 0004f31e37b1
Beeped 0004f31d4e81
Beeped 0004f3769028


ws error: Connection timed out
ws close


In [ ]:
# Plays midi file through robot
import netsblox
import time
import nblib
import mido

Client = netsblox.Client()
RoboScape = netsblox.RoboScape(client=Client)
print("Initialized RoboScape")
nb = nblib.nb(Client, RoboScape, "37ac")

mid = mido.MidiFile("midi/tetris.mid")
note_events = []
for msg in mid:
    if msg.type == "note_on" and msg.velocity > 0:
        freq = nb.music.midi_to_freq(msg.note)
        duration = msg.time
        if duration < 0.1:
            continue
        note_events.append((freq, duration * 2))

while not nb.this.connected():
    time.sleep(1)
input("ready")
for freq, dur in note_events:
    nb.this.beep(int(dur * 1000), int(freq))

In [12]:
# Plays midi file using 2 robots
import netsblox
import time
import nblib
import mido
import threading

Client = netsblox.Client()
RoboScape = netsblox.RoboScape(client=Client)
print("Initialized RoboScape")
robot1 = nblib.nb(Client, RoboScape, "37ac")
robot2 = nblib.nb(Client, RoboScape, "4e81")

midis = [mido.MidiFile("midi/maybe.mid"), mido.MidiFile("midi/maybe2.mid")]
low_notes = []
high_notes = []

for mid in midis:
    for msg in mid:
        if msg.type == "note_on" and msg.velocity > 0:
            freq = robot1.music.midi_to_freq(msg.note)
            duration = msg.time
            if msg.note < 60:
                low_notes.append((freq, duration))
            else:
                high_notes.append((freq, duration))

print(low_notes)
print(high_notes)

while not robot1.this.connected() and robot2.this.connected():
    time.sleep(1)
input("ready")
def play_notes(robot,notes):
    for freq, dur in notes:
        robot.this.beep(int(dur * 1000), int(freq))

# threading.Thread(target=play_notes, args=(robot1, low_notes)).start()
threading.Thread(target=play_notes, args=(robot2, high_notes)).start()

Initialized RoboScape
[(233.08188075904496, 0.008333333333333333), (246.94165062806206, 0.19583333333333333), (233.08188075904496, 0), (220.0, 0.19583333333333333), (184.9972113558172, 0), (195.99771799087463, 0.0125), (233.08188075904496, 0), (233.08188075904496, 0.19583333333333333), (220.0, 0), (207.65234878997256, 0.0125), (207.65234878997256, 0.004166666666666667), (233.08188075904496, 0.19166666666666665), (207.65234878997256, 0), (233.08188075904496, 0.39166666666666666), (184.9972113558172, 0.016666666666666666), (41.20344461410875, 0), (82.4068892282175, 0.004166666666666667), (41.20344461410875, 0.19583333333333333), (82.4068892282175, 0), (41.20344461410875, 0.19583333333333333), (82.4068892282175, 0), (41.20344461410875, 0), (82.4068892282175, 0.19583333333333333), (55.0, 0), (110.0, 0.0125), (55.0, 0.19583333333333333), (110.0, 0), (55.0, 0.19583333333333333), (110.0, 0), (55.0, 0), (110.0, 0.19583333333333333), (51.91308719749314, 0), (103.82617439498628, 0.0125), (51.913

ws error: Connection timed out
ws close


In [17]:
import netsblox
import time
import nblib
import threading

Client = netsblox.Client()
RoboScape = netsblox.RoboScape(client=Client)
print("Initialized RoboScape")
robot1 = nblib.nb(Client, RoboScape, "37ac")
robot2 = nblib.nb(Client, RoboScape, "4e81")

notes1 = [261.63, 293.66, 329.63, 349.23, 392.00, 440.00, 493.88, 523.25]
notes2 = [261.63, 293.66, 329.63, 349.23, 392.00, 440.00, 493.88, 523.25]
notes2.reverse()

while not robot1.this.connected() and robot2.this.connected():
    time.sleep(1)
input("ready")
def play_notes(robot,notes):
    for freq in notes:
        robot.this.beep(250, int(round(freq)))

threading.Thread(target=play_notes, args=(robot1, notes1)).start()
threading.Thread(target=play_notes, args=(robot2, notes2)).start()

Initialized RoboScape


ws error: Connection timed out
ws close


In [ ]:
import netsblox
import nblib
import random
import keyboard
import pygame
import pygame_gui

Client = netsblox.Client()
RoboScape = netsblox.RoboScape(client=Client)
print("Initialized RoboScape")
nb = nblib.nb(Client, RoboScape, "37ac")

def getRobots():
    i = 0
    robots = nb.getRobots()
    for robot in robots:
        i += 1
        print(f"{i}: {robot}")
    return robots

getRobots()

# init movement
keyboard.on_press_key("up", lambda: nb.this.setSpeed(40, 40))
keyboard.on_release_key("up", lambda: nb.this.stop())
keyboard.on_press_key("down", lambda: nb.this.setSpeed(-40, -40))
keyboard.on_release_key("down", lambda: nb.this.stop())
keyboard.on_press_key("right", lambda: nb.this.setSpeed(20, -20))
keyboard.on_release_key("right", lambda: nb.this.stop())
keyboard.on_press_key("left", lambda: nb.this.setSpeed(-20, 20))
keyboard.on_release_key("left", lambda: nb.this.stop())

# init gui
pygame.init()
pygame.display.set_caption("netsblox menu")
window_surface = pygame.display.set_mode((800, 600))
background = pygame.Surface((800, 600))
background.fill(pygame.Color("#000000"))
manager = pygame_gui.UIManager((800, 600))

ScanAllButton = pygame_gui.elements.UIButton(relative_rect=pygame.Rect(0, 10, 75, 75), text="ScanAll", manager=manager)
BeepAllButton = pygame_gui.elements.UIButton(relative_rect=pygame.Rect(0, 95, 75, 75), text="BeepAll", manager=manager)
BeepOthersButton = pygame_gui.elements.UIButton(relative_rect=pygame.Rect(0, 180, 75, 75), text="BeepOthers", manager=manager)
BeepMineButton = pygame_gui.elements.UIButton(relative_rect=pygame.Rect(0, 265+85, 75, 75), text="BeepMine", manager=manager)
ListenAllButton = pygame_gui.elements.UIButton(relative_rect=pygame.Rect(0, 265+85+85, 75, 75), text="ListenAll", manager=manager)

@Client.on_message(nb.message.robotCommand, nb.message.robotMessage)
def y(robot, command): print(f"{robot}: {command}")

clock = pygame.time.Clock()
is_running = True
while is_running:
    time_delta = clock.tick(60) / 1000
    for event in pygame.event.get():
        if event.type == pygame.QUIT: is_running = False
        if event.type == pygame_gui.UI_BUTTON_PRESSED:
            if event.ui_element == ScanAllButton:
                robots = getRobots()
            if event.ui_element == BeepAllButton:
                for robot in nb.getRobots():
                    nb.beep(robot, 100, random.randint(100, 4000))
            if event.ui_element == BeepOthersButton:
                for robot in nb.getOthers():
                    nb.beep(robot, 100, random.randint(100, 4000))
            if event.ui_element == BeepMineButton:
                nb.this.beep(100, 300)
            if event.ui_element == ListenAllButton:
                nb.roboscape.listen(nb.roboscape.get_robots())
        manager.process_events(event)
    manager.update(time_delta)
    window_surface.blit(background, (0, 0))
    manager.draw_ui(window_surface)
    pygame.display.update()
pygame.quit()